In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os
import shutil

print("🚀 Initialisation (Mode Nettoyage & Réparation)...")

# --- 1. NETTOYAGE PRÉALABLE ---
# On supprime les anciens dossiers pour éviter les conflits
if os.path.exists('flower_photos'):
    shutil.rmtree('flower_photos')
    print("🧹 Anciens fichiers supprimés.")

# --- 2. TÉLÉCHARGEMENT MANUEL (WGET) ---
# On télécharge directement à la racine
!wget -q https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz
# On décompresse
!tar -xzf flower_photos.tgz

# On définit le dossier cible
data_dir = 'flower_photos'

# --- 3. VÉRIFICATION CRUCIALE ---
# On regarde ce qu'il y a dedans. On DOIT voir : daisy, dandelion, roses, etc.
contenu = os.listdir(data_dir)
print(f"🧐 CONTENU DU DOSSIER : {contenu}")

if 'roses' not in contenu:
    raise ValueError("ERREUR : Les dossiers de fleurs sont introuvables !")
else:
    print("✅ Structure des dossiers VALIDÉE (5 Classes trouvées) !")

# --- 4. PRÉPARATION ---
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# On charge les données
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(160, 160),
    batch_size=32,
    class_mode='sparse', # Mode multi-classes robuste
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(160, 160),
    batch_size=32,
    class_mode='sparse',
    subset='validation'
)

# --- 5. MODÈLE ---
base_model = MobileNetV2(input_shape=(160, 160, 3),
                         include_top=False,
                         weights='imagenet')
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(5, activation='softmax')(x) # 5 Classes obligatoires

model = Model(inputs=base_model.input, outputs=predictions)

# --- 6. ENTRAÎNEMENT ---
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print("🌻 Démarrage de l'entraînement...")
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator
)

# --- 7. SAUVEGARDE ---
model.save('agro_scan_model.h5')
print("✅ Modèle sauvegardé sous 'agro_scan_model.h5'")
print("👉 CLASSES FINALES :", train_generator.class_indices)

🚀 Initialisation (Mode Nettoyage & Réparation)...
🧐 CONTENU DU DOSSIER : ['sunflowers', 'roses', 'daisy', 'LICENSE.txt', 'dandelion', 'tulips']
✅ Structure des dossiers VALIDÉE (5 Classes trouvées) !
Found 2939 images belonging to 5 classes.
Found 731 images belonging to 5 classes.
🌻 Démarrage de l'entraînement...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 62s 486ms/step - accuracy: 0.3790 - loss: 1.5479 - val_accuracy: 0.6676 - val_loss: 0.8446
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 27s 299ms/step - accuracy: 0.7122 - loss: 0.7972 - val_accuracy: 0.7729 - val_loss: 0.6405
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 26s 286ms/step - accuracy: 0.7811 - loss: 0.6142 - val_accuracy: 0.7948 - val_loss: 0.5701
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 26s 288ms/step - accuracy: 0.8107 - loss: 0.5331 - val_accuracy: 0.8112 - val_loss: 0.5288
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 26s 288ms/step - accuracy: 0.8309 - loss: 0.4813 - val_accuracy: 0.8331 - val_loss: 0.4826


✅ Modèle sauvegardé sous 'agro_scan_model.h5'
👉 CLASSES FINALES : {'daisy': 0, 'dandelion': 1, 'roses': 2, 'sunflowers': 3, 'tulips': 4}
